In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
 %pip install idx2numpy  #To convert standard IDX format MNIST by "yan Lacun" into numpy array

  Created wheel for idx2numpy: filename=idx2numpy-1.2.3-py3-none-any.whl size=7918 sha256=35dc0762f549cb905974df67d5a0b7507347e7d94ceaad88aa91d94383bedb4e
  Stored in directory: /root/.cache/pip/wheels/1a/ce/ad/d5e95a35cfe34149aade5e500f2edd535c0566d79e9a8e1d8a
Successfully built idx2numpy


In [ ]:
 
import pandas
from keras.models import Sequential #swquential is for plain stack of layers with each having one i/p and one o/p
from keras.layers import Dense
#from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
import idx2numpy
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA
 
## Importing the dataset present in the drive into a numpy array
# Importing the 112800 training images (28x28)
X_train = idx2numpy.convert_from_file('/content/drive/MyDrive/PRML/Project/MNIST_data/gzip/emnist-balanced-train-images-idx3-ubyte')### 60k training images each with 784 pixel elements 
# Importing the 112800 training labels
Y_train = idx2numpy.convert_from_file('/content/drive/MyDrive/PRML/Project/MNIST_data/gzip/emnist-balanced-train-labels-idx1-ubyte')#### labels--digits for each training image among 60k
# Importing the 18800 testing images (28x28)
X_test=idx2numpy.convert_from_file('/content/drive/MyDrive/PRML/Project/MNIST_data/gzip/emnist-balanced-test-images-idx3-ubyte')
# Importing the 18800 testing Labels
Y_test=idx2numpy.convert_from_file('/content/drive/MyDrive/PRML/Project/MNIST_data/gzip/emnist-balanced-test-labels-idx1-ubyte')
 
 
print('\ntrain images:',X_train.shape)
print('train labels:',Y_train.shape)
print('test images:',X_test.shape)
print('test labels:',Y_test.shape)


train images: (112800, 28, 28)
train labels: (112800,)
test images: (18800, 28, 28)
test labels: (18800,)


In [ ]:
X_train=X_train.reshape((X_train.shape[0],28,28,1)).astype('float32')
X_test=X_test.reshape((X_test.shape[0],28,28,1)).astype('float32')

In [ ]:
 y_1hot=np_utils.to_categorical(Y_train)
print(y_1hot)
y_1hotTest=np_utils.to_categorical(Y_test)

[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
layers=[0,1,2,3,4,5]
filters=[64,32,16,8,4]
Tacrcy=[];
Vacrcy=[];
e=10
for l in layers:
         
        model=Sequential()
        #initial layer
        model.add(Conv2D(128,kernel_size=(3,3),input_shape=(28,28,1),activation='relu'))
        
        ##stack of subsequrnt layers 
        if l>0:
            for i in range(0,int(l)):
                model.add(Conv2D(filters[i],kernel_size=(3,3),activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='valid'))
     
        model.add(Flatten())
 
        model.add(Dense(15000,activation='relu',name='i/p-layer'))
        model.add(Dense(47,activation='softmax',name='o/p-layer'))
 
        model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
        hit=model.fit(X_train,y_1hot,epochs=e,validation_split=0.33,batch_size=64,verbose=1)
        k=hit.history
        Tacrcy.append(k['accuracy'][e-1])
        Vacrcy.append(k['val_accuracy'][e-1])

In [ ]:
print(Tacrcy,Vacrcy)

In [ ]:
#Tacrcy=[0.8152:(128,64,32),0.6528:(128,64)(35min),0.9070724248886108:(128)]
#Vacrcy=[0.8015:(128,64,32),0.6813:(128,64),0.8172733187675476:(128)]

In [ ]:
l=[1,2,3,4,5,6]
plt.figure(1)
plt.plot(l,Tacrcy,color='g',marker='o',linestyle='dashed')
plt.plot(l,Vacrcy,color='r',marker='o',linestyle='dashed')
plt.xlabel('No.of Conv Layers')
plt.ylabel('accuracy')
plt.title('Kernels:4,8,16,32,64,128')
plt.legend(['Training','Validation'])
plt.grid()